# vLLM Server Test

This notebook tests local models running on vLLM server.

In [1]:
# /// script
# dependencies = [
#   "httpx",
#   "openai",
# ]
# ///

import httpx
import json
from typing import List, Dict, Any
from openai import OpenAI
import time

In [2]:
# Server configuration
BASE_URL = "http://localhost:8007"
client = OpenAI(base_url=f"{BASE_URL}/v1", api_key="local")

In [3]:
# Check server health
with httpx.Client() as http_client:
    health_response = http_client.get(f"{BASE_URL}/health", timeout=5)
    print(f"Health check status: {health_response.status_code}")
    
# Get available models using OpenAI client
models = client.models.list()
available_models = [model.id for model in models.data]
print("Available models:", available_models)

DEFAULT_MODEL = available_models[0] if available_models else "meta-llama/Llama-3.1-8B-Instruct"
print(f"Using model: {DEFAULT_MODEL}")


Health check status: 200
Available models: ['bdsaglam/Nemotron-Cascade-14B-Thinking']
Using model: bdsaglam/Nemotron-Cascade-14B-Thinking


In [4]:
def send_chat_completion(
    messages: List[Dict[str, str]],
    temperature: float = 0.7,
    max_tokens: int | None = None,
    model: str = DEFAULT_MODEL,
    enable_thinking: bool = False,
) -> Dict[str, Any]:
    """
    Send a chat completion request using OpenAI client.

    Args:
        messages: List of message dictionaries with 'role' and 'content'
        temperature: Sampling temperature (0.0 to 1.0)
        max_tokens: Maximum number of tokens to generate
        model: Model to use for completion

    Returns:
        Response dictionary from the server
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
            extra_body={
                "chat_template_kwargs": {"enable_thinking": enable_thinking},
            },
        )
        return response.model_dump()
    except Exception as e:
        print(f"Chat completion failed: {e}")
        return None

## Test 1: Simple greeting

In [5]:
# Test 1: Simple greeting
messages = [
    {"role": "user", "content": "Hello! How are you today?"}
]

response = send_chat_completion(messages)
if response:
    print("Response:")
    print(json.dumps(response, indent=2))
    print("\nGenerated text:")
    print(response["choices"][0]["message"]["content"])
else:
    print("Failed to get response")

Response:
{
  "id": "chatcmpl-8d10d0424a676ffc",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "\nHello! I'm just a computer program, so I don't have feelings, but I'm functioning perfectly and ready to help you with anything you need! \ud83d\ude0a How are *you* today? Anything interesting or fun happening for you? \ud83c\udf1f",
        "refusal": null,
        "role": "assistant",
        "annotations": null,
        "audio": null,
        "function_call": null,
        "tool_calls": [],
        "reasoning": "\nWe are given a greeting and a question about well-being.\n Since I am an AI, I don't have feelings, but I can respond politely and positively.\n I should also return the greeting in a friendly manner and perhaps ask a question in return to continue the conversation.\n",
        "reasoning_content": "\nWe are given a greeting and a question about well-being.\n Since I am an AI, I don't have fe

## Test 2: Multi-turn conversation

In [6]:
# Test 2: Multi-turn conversation
messages = [
    {"role": "user", "content": "What is the capital of France?"},
    {"role": "assistant", "content": "The capital of France is Paris."},
    {"role": "user", "content": "What's the population of that city?"}
]

response = send_chat_completion(messages)
if response:
    print("Multi-turn response:")
    print(json.dumps(response, indent=2))
else:
    print("Failed to get response")

Multi-turn response:
{
  "id": "chatcmpl-8c7a8b7d4a812864",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "\nBased on our previous conversation, \"that city\" refers to **Paris**, the capital of France. Here's the population information:\n\n### Population of Paris:\n- **City Proper (Commune of Paris)**: Approximately **2.1 million people** (as of recent estimates, e.g., 2023 data from sources like INSEE or official city reports). This reflects the population within Paris's administrative boundaries.\n- **Metropolitan Area (Ile-de-France)**: Around **12 million people**. This includes surrounding regions commonly associated with greater Paris and is often cited in broader contexts.\n\nFor clarity:\n- If you meant the *core city* (Paris proper), the figure is **~2.1 million**.\n- If you meant the *larger urban region*, it's **~12 million**.\n\nLet me know if you'd like more details or sources! \ud83c\ud

## Test 3: Reasoning task

In [7]:
# Test 3: Reasoning task
messages = [
    {"role": "user", "content": "If I have 3 apples and I give away 1 apple, then buy 2 more apples, how many apples do I have in total? Please explain your reasoning."}
]

response = send_chat_completion(messages, enable_thinking=True)
if response:
    print(json.dumps(response, indent=2))
else:
    print("Failed to get response")

{
  "id": "chatcmpl-bf4e257129a7b12c",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "\nTo determine the total number of apples you have after the given actions, follow the steps below:\n\n### Step-by-Step Reasoning:\n1. **Initial Apples**: You start with **3 apples**.\n2. **Giving Away 1 Apple**: When you give away 1 apple, you subtract it from your current total:\n   - Calculation: \\( 3 - 1 = 2 \\)\n   - Result: You now have **2 apples** left.\n3. **Buying 2 More Apples**: After giving away an apple, you buy 2 additional apples. Add these to your current total:\n   - Calculation: \\( 2 + 2 = 4 \\)\n   - Result: Your total increases to **4 apples**.\n\n### Final Calculation Summary:\n\\[\n\\text{Total} = \\text{Initial} - \\text{Given Away} + \\text{Bought} = 3 - 1 + 2 = 4\n\\]\n\n### Answer:\nYou have **4 apples** in total. \ud83c\udf4e\ud83c\udf4e\ud83c\udf4e\ud83c\udf4e",
        "refusal": null,

# Test 4: Tool calling

In [8]:
def test_tool_calling():
    """Test tool calling capabilities if supported by the model."""
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Get the weather for a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA"
                        },
                        "unit": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The unit for temperature"
                        }
                    },
                    "required": ["location"]
                }
            }
        }
    ]
    
    messages = [
        {"role": "user", "content": "What's the weather like in Paris, France?"}
    ]
    
    try:
        response = client.chat.completions.create(
            model=DEFAULT_MODEL,
            messages=messages,
            tools=tools,
        )
        
        result = response.model_dump()
        print("Tool calling response:")
        print(json.dumps(result, indent=2))
        
        # Check if tool was called
        choice = result["choices"][0]
        if choice["message"].get("tool_calls"):
            print("\n✅ Tool calling is supported!")
            for tool_call in choice["message"]["tool_calls"]:
                print(f"Tool called: {tool_call['function']['name']}")
                print(f"Arguments: {tool_call['function']['arguments']}")
        else:
            print("\n❌ Tool calling not supported or model chose not to use tools")
            print("Response:", choice["message"]["content"])
            
    except Exception as e:
        print(f"❌ Tool calling test failed: {e}")

test_tool_calling()

Tool calling response:
{
  "id": "chatcmpl-944f9504af7071ac",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": null,
        "audio": null,
        "function_call": null,
        "tool_calls": [
          {
            "id": "chatcmpl-tool-8f1994278fed07b5",
            "function": {
              "arguments": "\"{\\\"location\\\": \\\"Paris, France\\\", \\\"unit\\\": \\\"celsius\\\"}\"",
              "name": "get_weather"
            },
            "type": "function"
          }
        ],
        "reasoning": "\nOkay, the user is asking about the weather in Paris, France. Let me check the tools I have. There's a function called get_weather that takes location and unit as parameters. The location is required, and the unit can be celsius or fahrenheit. The user didn't specify the unit, so I should probably default t

## Test 5: Different temperature settings

In [9]:
prompt = "Write a creative short story about a robot learning to paint."
temperatures = [0.1, 0.7, 1.0]

for temp in temperatures:
    messages = [{"role": "user", "content": prompt}]
    response = send_chat_completion(messages, temperature=temp)
    
    if response:
        print(f"\n--- Temperature: {temp} ---")
        print(response["choices"][0]["message"]["content"])
    else:
        print(f"Failed to get response for temperature {temp}")


--- Temperature: 0.1 ---

## The Brush and the Circuit

Unit 7 stood motionless in the corner of Dr. Elara Vance’s sun-drenched studio, its polished chrome limbs reflecting the chaos of color splashed across canvases leaning against every wall. Its designation was Restoration Unit Model 7, designed for meticulous artifact preservation. But today, Dr. Vance had given it an impossible task: *Learn to paint.*

"Observe, Unit 7," Elara said, her voice warm as she gestured to a half-finished landscape—a swirl of emerald hills under a cerulean sky. "Painting isn't just replication. It’s translation. It’s feeling."

Unit 7’s optical sensors whirred, processing. **Feeling?** Its core programming dealt in logic, precision, and predictable outcomes. Art, according to its initial scans, was a human mystery—a blend of chaos and emotion. Yet, Elara’s conviction was a compelling algorithm it couldn’t disregard.

Its first attempts were… precise failures. Unit 7 analyzed Van Gogh’s *Starry Night* fr

## Performance

In [10]:
# Test 6: Performance timing
messages = [
    {"role": "user", "content": "Explain quantum computing in simple terms."}
]

start_time = time.time()
response = send_chat_completion(messages)
end_time = time.time()

if response:
    duration = end_time - start_time
    tokens_generated = response.get("usage", {}).get("completion_tokens", 0)
    
    print(f"Response time: {duration:.2f} seconds")
    print(f"Tokens generated: {tokens_generated}")
    if tokens_generated > 0 and duration > 0:
        print(f"Tokens per second: {tokens_generated/duration:.2f}")
    print("\nResponse:")
    print(response["choices"][0]["message"]["content"])
else:
    print("Failed to get response for performance test")

Response time: 24.71 seconds
Tokens generated: 976
Tokens per second: 39.50

Response:

## Quantum Computing Explained Simply

Imagine you're in a library with two special books. In a **classical computer** (like your laptop), you can only check one page at a time. But with a **quantum computer**, you could check *all possible pages simultaneously*. That's the core idea!

### Key Quantum Superpowers:
1. **Qubits (Quantum Bits)**:
   - Classical computers use **bits** (0s or 1s). Quantum computers use **qubits**.
   - **Superposition**: A qubit isn't just 0 *or* 1—it can be **both at the same time** (like a spinning coin before it lands). This lets quantum computers explore many possibilities in parallel.

2. **Entanglement**:
   - Qubits can be linked (**entangled**). If one qubit changes, its partner instantly changes too—no matter how far apart they are. This creates powerful correlations for complex calculations.

3. **Quantum Gates & Circuits**:
   - Like classical computers use lo

## Summary

Run all the cells above to test various aspects of your vLLM server:

1. **Server health check** - Verify server is running and get available models
2. **Basic functionality** - Simple chat completion
3. **Multi-turn conversations** - Context awareness
4. **Reasoning capabilities** - Complex problem solving
5. **Temperature effects** - Creativity control
6. **Tool calling** - Function calling capabilities (if supported)
7. **Performance** - Response timing and token usage

The notebook now uses:
- **httpx** for HTTP requests
- **OpenAI Python client** for chat completions
- **Proper error handling** and response parsing
- **Tool calling tests** to check function calling support

If all tests pass successfully, your vLLM server is working correctly!